In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-review-dataset/imdb_master.csv
/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip


In [2]:
import pandas as pd
df1 = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", delimiter = "\t")
df1 = df1.drop(['id'], axis=1)
df1.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
df2 = pd.read_csv("/kaggle/input/imdb-review-dataset/imdb_master.csv", encoding = "latin-1")
df2 = df2[['review','label']]
df2 = df2.rename(columns = {'label': 'sentiment'})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [4]:
df2 = df2[df2['sentiment'] != 'unsup']
df2['sentiment'] = df2['sentiment'].map({"neg" : 1, "pos" : 0})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,1
1,This is an example of why the majority of acti...,1
2,"First of all I hate those moronic rappers, who...",1
3,Not even the Beatles could write songs everyon...,1
4,Brass pictures (movies is not a fitting word f...,1


In [5]:
df = pd.concat([df1,df2]).reset_index(drop=True)
df.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [6]:
'''
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
'''

"\nimport nltk\nnltk.download('stopwords')\nnltk.download('wordnet')\nnltk.download('averaged_perceptron_tagger_eng')\n"

In [ ]:
'''
1번째 방식(기존 코드)
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))
'''

In [7]:
'''
2번째 방식(pos_tag)
from nltk import pos_tag
from nltk.corpus import wordnet,stopwords
import re
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tagged = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged]
    return ' '.join(lemmatized)
# 기존 방식 f1_score = 0.96
# pos-tagging 방식 f1_score = 0.34
'''

'\nfrom nltk import pos_tag\nfrom nltk.corpus import wordnet,stopwords\nimport re\nfrom nltk.stem import WordNetLemmatizer\n\nstop_words = set(stopwords.words("english"))\nlemmatizer = WordNetLemmatizer()\n\ndef get_wordnet_pos(tag):\n    if tag.startswith(\'J\'):\n        return wordnet.ADJ\n    elif tag.startswith(\'V\'):\n        return wordnet.VERB\n    elif tag.startswith(\'N\'):\n        return wordnet.NOUN\n    elif tag.startswith(\'R\'):\n        return wordnet.ADV\n    else:\n        return wordnet.NOUN\n\ndef clean_text(text):\n    text = re.sub(r\'[^\\w\\s]\', \'\', text)\n    text = text.lower()\n    tokens = text.split()\n    tokens = [word for word in tokens if word not in stop_words]\n    tagged = pos_tag(tokens)\n    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged]\n    return \' \'.join(lemmatized)\n# 기존 방식 f1_score = 0.96\n# pos-tagging 방식 f1_score = 0.34\n'

In [8]:
#spacy 방식
import spacy
import re

nlp = spacy.load("en_core_web_sm")
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def clean_text_spacy(text):
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [9]:
df['processed_reviews'] = df.review.apply(lambda x: clean_text_spacy(x))
df.head()

,sentiment,review,processed_reviews
0,1,With all this stuff going down at the moment w...,stuff go moment mj ve start listen music watch...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertaining f...
2,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell give welcome ...
3,0,It must be assumed that those who praised this...,assume praise film great filmed opera not read...
4,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


In [10]:
df.processed_reviews.apply(lambda x: len(x.split(" "))).mean()

106.81069333333333

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D  # Convolution1D은 Conv1D로 대체

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['processed_reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['processed_reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

2025-07-23 05:33:03.615431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753248783.811405      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753248783.865847      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753248804.065089      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1753248804.065827      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

Epoch 1/3


I0000 00:00:1753248809.820805      85 cuda_dnn.cc:529] Loaded cuDNN version 90300


600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - accuracy: 0.5853 - loss: 0.6782 - val_accuracy: 0.2335 - val_loss: 0.8454
Epoch 2/3
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.6285 - loss: 0.6503 - val_accuracy: 0.2583 - val_loss: 1.0149
Epoch 3/3
600/600 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6601 - loss: 0.6209 - val_accuracy: 0.2675 - val_loss: 0.9733


In [12]:
df_test=pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip",header=0, delimiter="\t", quoting=3)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x: clean_text_spacy(x))
df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = df_test["sentiment"]
list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step
F1-score: 0.28881201184065053
Confusion matrix:


array([[ 1010,  8451],
       [11490,  4049]])

In [ ]:
# 기존 방식 f1_score = 0.96
# pos-tagging 방식 f1_score = 0.34
# spacy 전처리 방식 f1-score : 0.28